# Import Library

Library yang digunakan pada notebook ini adalah"
- pandas (operasi sederhana pada dataset)
- sklearn (modul machine learning)

In [78]:
import pandas as pd
from sklearn import datasets, model_selection
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

## Load Datasets

In [107]:
cancer = datasets.load_breast_cancer(as_frame=True)
dtcancer = cancer.data
dttennis = pd.read_csv("../data/play_tennis.csv")


0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Name: target, Length: 569, dtype: int64

In [80]:
def simpleDescribe(df: datasets):
    print('Jumlah kolom: ' + str(len(df.columns)))
    print('Jumlah baris: ' + str(len(df)))
    print()
    print('Kolom pada tabel:')
    print(df.columns.tolist())


### Data cancer

Sampel data kanker payudara yang disediakan oleh data internal sklearn:

In [95]:
dtcancer.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [82]:
simpleDescribe(dtcancer)

Jumlah kolom: 30
Jumlah baris: 569

Kolom pada tabel:
['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']


### Data Tennis
Sampel data tennis yang didapatkan dari file eksternal pada folder data: 

In [83]:
dttennis.head()

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes


In [84]:
simpleDescribe(dttennis)

Jumlah kolom: 6
Jumlah baris: 14

Kolom pada tabel:
['day', 'outlook', 'temp', 'humidity', 'wind', 'play']


Terdapat beberapa hal penting yang bisa dilihat disini, yaitu:
1. Kolom 'day' pada dataset merupakan data hari ke-sekian dan merupakan urutan hari dalam minggu. Untuk bisa mengetahui pola atau pengaruh 'urutan hari' dari sebuah tabel terhadap play adalah dengan melakukan operasi modulo 7 untuk melihat pengaruh hari (senin, selasa, dll) terhadap atribut play.
2. Data pada dataset tersebut masih berupa string dan perlu untuk di-encode sesuai dengan kategorinya

In [85]:
for i, data in dttennis['day'].iteritems():
    dttennis['day'][i] = int(data[1:])%7
dttennis.head(2)

,day,outlook,temp,humidity,wind,play
0,1,Sunny,Hot,High,Weak,No
1,2,Sunny,Hot,High,Strong,No


#### Encode Play Tennis Column

In [86]:
le = LabelEncoder()
dttennis_old = dttennis.copy()
for col in dttennis_old[1:]:
    dttennis[col] = le.fit_transform(dttennis_old[col])
dttennis.head()

,day,outlook,temp,humidity,wind,play
0,1,2,1,0,1,0
1,2,2,1,0,0,0
2,3,0,1,0,1,1
3,4,1,2,0,1,1
4,5,1,0,1,1,1


## Split Dataset

Memisahkan dataset menjadi 2 bagian yaitu train dan test

In [87]:
ctrain, ctest = model_selection.train_test_split(dtcancer, test_size=0.2, train_size=0.8, random_state=1)
tntrain, tntest = model_selection.train_test_split(dttennis, test_size=0.2, train_size=0.8, random_state=1)

Memisahkan dataset menjadi 4 bagian:
- **x train:** feature yang akan digunakan untuk train model
- **y train:** ground truth (hasil asli)
- **x test:** feature yang digunakan untuk diuji oleh model
- **y test:** data yang digunakan untuk memvalidasi hasil prediksi

### Dataset Cancer

In [88]:
y = dtcancer['worst fractal dimension']
x = dtcancer.drop('worst fractal dimension', axis=1)

x_ctrain, x_ctest, y_ctrain, y_ctest = model_selection.train_test_split(x, y, test_size=0.2, train_size=0.8, random_state=1)

### Dataset Tennis

In [89]:
y = dttennis['play']
x = dttennis.drop('play', axis=1)

x_tntrain, x_tntest, y_tntrain, y_tntest = model_selection.train_test_split(x, y, test_size=0.2, train_size=0.8, random_state=1)

# Pembelajaran Mesin

## Decision Tree Classifier

**Decision Tree Classifier** adalah sebuah metode supervised learning yang digunakan untuk memberikan model yang akan membagi data bergantung dari parameter tertentu.

Tahap pembuatan DTC adalah:
1. Pilih atribut terbaik menggunakan ASM (Attribute Selection Measures)
2. Buat atribut tersebut menjadi decision node dan pecah dataset menjadi lebih kecil
3. Ulangi secara rekursif hingga kondisi berikut terpenuhi:
    - semua atribut sudah habis
    - tidak ada instansi lain lagi
    - semua tuple terdapat dalam nilai atribut yang sama

Banyak metode ASM yang bisa digunakan, tetapi yang akan dipakai disini adalah *Information Gain*

### Breast Cancer Dataset

In [91]:
clf_c = DecisionTreeClassifier()
clf_c = clf_c.fit(x_ctrain,y_ctrain)
y_cpred = clf_c.predict(x_ctest)
print('Accuracy: ', accuracy_score(y_ctest,y_cpred))

ValueError: Unknown label type: 'continuous'

### Tennis Dataset

## Id3 Estimator

Penjelasan Id3 Estimator

In [ ]:
# Kode Id3 Estimator

## KMeans

Penjelasan KMeans

In [6]:
ckmeans = KMeans(n_clusters=2, random_state=42).fit(ctrain)
ckmeanspredict = ckmeans.predict(ctest)

tnkmeans = KMeans(n_clusters=2, random_state=42).fit(tntrain)
tnkmeanspredict = tnkmeans.predict(tntest)

# accuracy_score(ckmeans.labels_, ckmeanspredict, normalize=False)
# f1_score(ckmeans.labels_, ckmeanspredict)

## Logistic Regression

Penjelasan Logistic Regression

In [75]:
# Kode Log regression

## Neural Network

Penjelasan Neural Network

In [7]:
clf = MLPClassifier(random_state=1, max_iter=2000)

# cNeural = clf.fit(x_ctrain, y_ctrain)
# cNeuralpredict = cNeural.predict(x_ctest)

tnNeural = clf.fit(x_tntrain, y_tntrain)
tnNeuralpredict = tnNeural.predict(x_tntest)

tnNeural.score(x_tntest, y_tntest)

0.3333333333333333

## SVM

Penjelasan SVM

In [8]:
# to do tomorrow
svc = SVC(C=1, kernel='linear')

# c_svc = svc.fit(x_ctrain, y_ctrain)
# c_svcpredict = c_svc.predict(x_ctest)

tn_svc = svc.fit(x_tntrain, y_tntrain)
tn_svcpredict = tn_svc.predict(x_tntest)